<a href="https://colab.research.google.com/github/SIDHARTH06/AI_CLUB_LOGO_GEN_GAN/blob/main/AIproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import time
from google.colab import drive
import h5py
drive.mount('/content/gdrive')
import keras
from keras.layers import Input,Dense,Reshape,Flatten
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential,Model
from keras.optimizers import Adam

img_rows=32
img_cols=32
channels=3
img_shape=(img_rows,img_cols,channels)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
hdf5_path="/content/gdrive/My Drive/LLD-icon.hdf5"
dataset=h5py.File(hdf5_path,"r")

In [ ]:
images, labels = (dataset['data'], dataset['labels/resnet/rc_64'])



In [ ]:
def build_generator():
  noise_shape=(100,)
  model=Sequential()
  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Dense(1024))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Dense(np.prod(img_shape),activation='tanh'))
  model.add(Reshape(img_shape))

  model.summary()
  noise=Input(shape=noise_shape)
  img=model(noise)
  
  return Model(noise,img)

In [ ]:
def build_discriminator():
  model.add(Flatten(input_shape=img_shape))
  model=Sequential()
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dense(1,activation='sigmoid'))
  model.summary()
  
  img=Input(shape=img_shape)
  validity=model(img)
  return Model(img,validity)
  

In [61]:
def train(epochs,batch_size=128,save_interval=500):
  images=(images.astype(np.float32)-127.5)/127.5
  half_batch=int(batch_size/2)
  
  for epoch in range(epochs):
    idx=np.random.randint(0,img_shape[0],half_batch)
    imgs=images[idx]
    noise=np.random.normal(0,1,(half_batch,100))

    gen_imgs=generator.predict(noise)
    d_loss_real=discriminator.train_on_batch(imgs,np.ones((half_batch,1)))
    d_loss_fake=discriminator.train_on_batch(gen_imgs,np.zeroes((half_batch,1))) 

    d_loss=0.5*(np.add(d_loss_real,d_loss_fake)) 
    noise=np.random.normal(0,1,(batch_size,100))
    valid_y=np.array([1]*batch_size)
    g_loss=combined.train_on_batch(noise,valid_y)
    print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

    if epoch%save_interval==0:
      save_imgs(epoch)
       

In [60]:
def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()